# Fast Imaging Pulse Sequences

Modern MRI scanning relies heavily fast imaging pulse sequences, primarily echo-planar imaging (EPI) and multiple Spin-echo (RARE/FSE/TSE) methods.  These allow for multiple k-space lines to be acquired within a single TR.


## Learning Goals

1. Describe how images are formed
    * Describe how EPI works
    * Describe how data is acquired in FSE/TSE sequences
    * Understand what "spoiling" means
1. Understand the most popular pulse sequences and their acronyms
    * Describe how EPI works
    * Describe how FSE/TSE sequences work
    * Describe how fast gradient-echo sequences work
1. Manipulate MRI sequence parameters to improve performance
    * Understand how and when to accelerate with EPI
    * Understand how and when to accelerate with FSE/TSE
    * Understand how contrast changes in fast gradient-echo sequences
1. Identify artifacts and how to mitigate them
    * Identify EPI artifacts including distortion and T2*
    * Identify FSE/TSE artifacts include T2 blurring
    * Identify fast gradient-echo artifacts such as banding in bSSFP

## Multiple Spin-echo Sequences (RARE/FSE/TSE) 

These pulse sequences use multiple spin-echo refocusing pulses after a single exictation pulse to acquire multiple k-space lines to be acquired during the multiple spin-echoes that are formed.  This technique was originally called Rapid Acquisition with Relaxation Enhancement (RARE), and is known on various MRI scanners as fast spin-echo (FSE, GE Healthcare), turbo spin-echo (TSE, Siemens Healthineers), or turbo field-echo (TFE, Philips Healthcare).

This is the most common pulse sequence used in clinical practice.  This is because it is highly SNR efficient and can also be used to generate multiple contrasts.  They are particularly effective for T2 and PD-weighted imaging.

### Simplified Pulse Sequence Diagram

![Multiple spin-echo Pulse Sequence Diagram](images/pulse_sequence-diagram-fastspinecho-simple.png) 

### Sequence parameters

* Echo spacing ($t_{esp}$) - time between each spin-echo
* Echo train length (ETL) - number of spin-echoes with a repetition
* Effective TE ($TE_{eff}$) - the echo time when the data closest to the center of k-space is acquired.  

### Tradeoffs

Pros
* Fast
* SNR efficient
* Supports multiple contrasts by manipulating $TE_{eff}$

Cons
* Long echo train lengths can lead to T2 blurring or other distortions
* High SAR from repeated refocusing pulses

## Echo-planar Imaging (EPI)

These pulse sequences readout multiple k-space lines sequentially for faster imaging.  It is commonly used for diffusion-weighted imaging and fMRI.

### Pulse Sequence and K-space Diagram

![EPI Pulse Sequence](images/pulse_sequence-diagram-EPI.png)

### Sequence parameters

* Echo spacing ($t_{esp}$) - time between each readout and gradient-echo
* Echo train length (ETL) - number of readouts or k-space lines acquired within a repetition
* TE - the time when the data closest to the center of k-space is acquired.  

### Tradeoffs

Pros
* Extremely Fast, supporting single-shot imaging

Cons
* Susceptible to chemical shift and susceptibility displacement artifacts
* Sensitive to gradient fidelity artifacts (e.g. Nyquist ghosting)
* Long echo train lengths can lead to T2* blurring

## Fast gradient-echo sequences

The basic gradient-echo sequence is typically a spoiled gradient-echo sequence.  The sequence is called spoiled because the transverse magnetization is spoiled by a spoiler gradient before the next RF pulse.

### Spoiler or Crusher gradients

A large, unbalanced gradient will create dephasing of the transverse magnetization across the imaging voxels.  This effectively eliminates the signal.  However, the net magnetization is not truly eliminated, and this can be refocused by gradients.

### RF spoiling

The RF axis of rotation is changed every TR.  This reduces the chances of magnetization from previous TRs to become coherently excited.  Specific RF spoiling schemes, such as quadratic phase incrementation, is required for this to be effective.

### Types of fast gradient-echo sequences

Spoiled gradient-echo (SPGR)/fast low-angle shot (FLASH)
* Aims to full spoil transverse magnetization every TR
* Both gradient and RF spoiling are used
* Pure T1 weighted contrast

Gradient-recalled acquisition in the steady state (GRASS)/fast imaging with steady-state precession (FISP)
* Allows for residual transverse magnetization to be used in the next TR
* Refouces frequency and phase encoding gradients
* RF spoiling used
* Increases T2* contrast


Steady-state free precession (SSFP)/time-reversed fast imaging with steady-state precession (PSIF)
* Use gradients to refocus signals from previous TRs
* Gradient spoiling but refocused in later TRs
* Creates T2/T1 contrast

Balanced SSFP/TrueFISP
* Balanced gradients every TR
* No spoiling, all magnetization preserved each TR
* Creates T2/T1 contrast
* High SNR efficiency